# Week 1 - Retreiving and Preparing Text for Machines

This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze. This notebook outlines scraping text from the
web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Finally, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis.

For this notebook we will be using the following packages:

In [2]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud_2020 #pip install git+git://github.com/Computational-Content-Analysis-2020/lucem_illud_2020.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [3]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [4]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [5]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Content analysis - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xh8jfApAMFIAAJw8PzAAAADJ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":931112759,"wgRevisionId":931112759,"wgArticleId":473317,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles needing expert attention with no reason or talk parameter","Article

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

In [6]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia
document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dm


This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [7]:
#content_analysis_save = 'wikipedia_content_analysis.html'
with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

In [8]:
content_analysis_save

'wikipedia_content_analysis.html'

Now lets open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag.

In [9]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

Content analysis is a research method for studying documents and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.[1] One of the key advantages of using content analysis to analyse social phenomena is its non-invasive nature, in contrast to simulating social experiences or collecting survey answers.

Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of texts or artifacts which are assigned labels (sometimes called codes) to indicate the presence of interesting, meaningful pieces of content.[2][3] By systematically labeling the content of a set of texts, researchers can analyse patterns of content quantitatively using statistical methods, or use qualitative methods to analyse meanings of content within texts.

Computers are increasingly used in conten

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [10]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0   Content analysis is a research method for stud...
1   Practices and philosophies of content analysis...
2   Computers are increasingly used in content ana...
3   Content analysis is best understood as a broad...
4   The simplest and most objective form of conten...
5   A further step in analysis is the distinction ...
6   More generally, content analysis is research u...
7   By having contents of communication available ...
8   Robert Weber notes: "To make valid inferences ...
9   There are five types of texts in content analy...
10  Over the years, content analysis has been appl...
11  In recent times, particularly with the advent ...
12  Quantitative content analysis has enjoyed a re...
13  Content analysis can also be described as stud...
14  Holsti groups fifteen uses of content analysis...
15  He also places these uses into the context of ...
16  The following table shows fifteen uses of cont...
17  The process of the initi

Now we have a `DataFrame` containing all relevant text from the page ready to be
processed

If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

In [11]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<re.Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [12]:
contentPTags

[<p><b>Content analysis</b> is a research method for studying <a href="/wiki/Document" title="Document">documents</a> and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup> One of the key advantages of using content analysis to analyse social phenomena is its non-invasive nature, in contrast to simulating social experiences or collecting survey answers.
 </p>,
 <p>Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of <a href="/wiki/Text_(literary_theory)" title="Text (literary theory)">texts</a> or artifacts which are <a href="/wiki/Coding_(social_sciences)" title="Coding (social sciences)">assigned labels (sometimes called codes)</a> to indicate the presence of interesting, <a 

In [13]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [14]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

The tentative topic i want to analyze this quarter is the bias in academic research that is currently being promoted in a developing country, concretely, Peru. The biggest research contest that happens yearly from the last 20 years and has produced 700 research documents, is financed with resources from the International Development Research Center (IDRC) and many government institutions at the national level. I want to explore what are the profiles from the winning research projects, and what kind of discourse, towards equality, economic development and gender, for example, are producing.  

One of the subtopics that i also want to find out about is about the politics of citation. What are the citations that are being promoted in the peruvian academia sphere? For doing so, i want to first check the general discourse about race biases in citations in the following article:

In [15]:
url = 'https://www.insidehighered.com/advice/2018/04/27/racial-exclusions-scholarly-citations-opinion'
saving_html = 'saving_here.html'

req = requests.get(url)
soup = bs4.BeautifulSoup(req.text, 'html.parser')
contentParagraphs = []
contentPTags = soup.body.findAll('p')
for pTag in contentPTags:
    for_mod = pTag.text
    #print(for_mod)
    #for_mod = re.sub(r"<a href=.+</a>", r"",for_mod)
    #Eliminate text that is less than 80 characters
    for_mod = re.sub(r"^.{1,80}$", r"",for_mod)
    #Eliminate weird character
    for_mod = re.sub(r"\xa0", r"",for_mod)
    for_mod = re.sub(r"\u202f", r"",for_mod)
    if for_mod:
        contentParagraphs.append(for_mod)
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)  



                                       paragraph-text
0   The lack of citations of scholars of color fur...
1   Citation is political. Since Richard Delgado’s...
2   Racially biased citations patterns are by no m...
3   Much of the bias in citation patterns may be u...
4   Inequality is reproduced (and whiteness is ins...
5   The racial politics of citation have real effe...
6   When I think about citation patterns, and the ...
7   I anticipate objections to this argument, bein...
8   Second, this history of racial exclusion in ac...
9   Victor Ray is an assistant professor of sociol...
10  How to respond to new federal Title IX regulat...
11  Long-term look at return on investment reveals...



# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [16]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[]


We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [17]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number
0,The lack of citations of scholars of color fur...,https://en.wikipedia.org/wiki/Content_analysis,0
1,Citation is political. Since Richard Delgado’s...,https://en.wikipedia.org/wiki/Content_analysis,1
2,Racially biased citations patterns are by no m...,https://en.wikipedia.org/wiki/Content_analysis,2
3,Much of the bias in citation patterns may be u...,https://en.wikipedia.org/wiki/Content_analysis,3
4,Inequality is reproduced (and whiteness is ins...,https://en.wikipedia.org/wiki/Content_analysis,4
5,The racial politics of citation have real effe...,https://en.wikipedia.org/wiki/Content_analysis,5
6,"When I think about citation patterns, and the ...",https://en.wikipedia.org/wiki/Content_analysis,6
7,"I anticipate objections to this argument, bein...",https://en.wikipedia.org/wiki/Content_analysis,7
8,"Second, this history of racial exclusion in ac...",https://en.wikipedia.org/wiki/Content_analysis,8
9,Victor Ray is an assistant professor of sociol...,https://en.wikipedia.org/wiki/Content_analysis,9


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [18]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [19]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-text,source,paragraph-number,source-paragraph-number,source-paragraph-text
0,The lack of citations of scholars of color fur...,https://en.wikipedia.org/wiki/Content_analysis,0,None,None
1,Citation is political. Since Richard Delgado’s...,https://en.wikipedia.org/wiki/Content_analysis,1,None,None
2,Racially biased citations patterns are by no m...,https://en.wikipedia.org/wiki/Content_analysis,2,None,None
3,Much of the bias in citation patterns may be u...,https://en.wikipedia.org/wiki/Content_analysis,3,None,None
4,Inequality is reproduced (and whiteness is ins...,https://en.wikipedia.org/wiki/Content_analysis,4,None,None
5,The racial politics of citation have real effe...,https://en.wikipedia.org/wiki/Content_analysis,5,None,None
6,"When I think about citation patterns, and the ...",https://en.wikipedia.org/wiki/Content_analysis,6,None,None
7,"I anticipate objections to this argument, bein...",https://en.wikipedia.org/wiki/Content_analysis,7,None,None
8,"Second, this history of racial exclusion in ac...",https://en.wikipedia.org/wiki/Content_analysis,8,None,None
9,Victor Ray is an assistant professor of sociol...,https://en.wikipedia.org/wiki/Content_analysis,9,None,None



# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>



The following code takes the recursive approach as it collects the links and recursively navigates throught the web pages, following multiple levels of links. more than i would need to extract the information i would need to do my project, i still have to think about my crawler in a recursive way  

In [20]:
import urllib

def link_ok(url, limit):
    parsed_url = urllib.parse.urlparse(url)
    nloc = parsed_url.netloc
    ld = len(limit)
    trunc_nloc = nloc[-(ld+1):]
    if not (limit == nloc or (trunc_nloc == "." + limit)):
        return False
    
    (filename, ext) = os.path.splitext(parsed_url.path)
    return (ext == "" or ext == ".html")

def crawl(url,depth):
    list_crawl.append(url)
    request = requests.get(url)
    soup = bs4.BeautifulSoup(request.text, 'html.parser')
    tagLinks = soup.findAll('a', href=re.compile('.'), class_=False)
    
    #list_crawl.append(tag)
    if depth > 0:
        for link in tagLinks:
            link = link.get('href')
            link = urllib.parse.urljoin(url,link) 
            if link not in list_crawl:
                if link_ok(link, limit):
                    crawl(link, depth - 1) 

In [23]:

link = "http://www.cies.org.pe/es/investigaciones-promovidas-por-el-cies"
limit = "cies.org.pe"
list_crawl = []
#Here, i will crawl the web page of the academic research contest and will extract the links that are available.
#Right now, my crawler is NOT working correctly because i imposed a restriction of not to go to web pages outside of
#the main domain. I will work on improving this part of the code for a next delivery. 
crawl(link,5)

ConnectionError: HTTPConnectionPool(host='www.elecciones.cies.org.pe', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a1ef8f990>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

Following next, i also made an iterative crawler that helps me collect information about the research projects that were financed. The dataframe contains information about title of the project, the abstract (in english and spanish), the author's names, institutional affiliation and year of completion

In [24]:
link = "http://www.cies.org.pe/es/investigaciones-promovidas-por-el-cies"

def make_soup(link,ext_link):
    new_link = urllib.parse.urljoin(link,ext_link)
    request = requests.get(new_link)
    soup = bs4.BeautifulSoup(request.text, 'html.parser')
    return soup

def review_esp(soup,tag,class_):
    see = soup.findAll(tag, {"class": class_})
    if len(see)!=0:
    #see = soup.findAll("div", {"class": "field field-name-field-resumen field-type-text-long field-label-above"})
        return see[0].text
    else:
        return 'no info'

def crawl_page(link,page):
    abs_link = "http://www.cies.org.pe/"
    total_url = link+'?page='+ str(page)
    request = requests.get(total_url)
    soup = bs4.BeautifulSoup(request.text, 'html.parser')
    tabs = soup.findAll("table", {"class": "views-table cols-3"})
    dic =  {'research_l':[],
               'title_l':[],
               'autor_l':[],
               'year_l':[],
           'abstrac_esp':[],
           'abstract_eng':[],
           'research_center':[],
           'research_type': [],
            'research_time':[],
            'dl_link': []
           }
    for tab in tabs:    
        tagLinks = tab.find_all('a', href=re.compile('.'), class_=False)
        tagTitle = tab.find_all('a')
        tagAuthor = tab.find_all('td', {"class": "views-field views-field-field-autor"})
        tagYear = tab.find_all('td', {"class": "views-field views-field-field-fecha-inv"})
   
        for tl in tagLinks:
            link_name = tl.get('href')
            dic['research_l'].append(link_name)
            dic['title_l'].append(tl.text)
            soup2 = make_soup(link,link_name)
            rev_esp = review_esp(soup2,"div","field field-name-field-resumen field-type-text-long field-label-above")
            rev_eng = review_esp(soup2,"div","field field-name-field-abstract field-type-text-long field-label-above")
            center = review_esp(soup2,"div","views-field views-field-field-centro-")
            center = review_esp(soup2,"div","views-field views-field-field-centro-")
            link_dl = soup2.find_all("div", {"class": "views-field views-field-field-descargar-investigaci-n"})
            if len(link_dl)!=0:
                if len(link_dl[0].find_all('a'))!=0:
                    rel_link = link_dl[0].find_all('a')[0].get('href')
                    rel_link = urllib.parse.urljoin(abs_link,rel_link)
            else:
                rel_link = "no link"

            dic['dl_link'].append(rel_link)
            #clearning center
            center = re.sub('Centro:','',center)
            center = center.strip()
            time_done = review_esp(soup2,"div","views-field views-field-field-tiempo-de-elaboraci-n")
            time_done = re.sub('Tiempo de elaboración:','',time_done)
            time_done = time_done.strip()
            kind_done = review_esp(soup2,"div","views-field views-field-field-tipo-de-investigaci-n")
            kind_done = re.sub('Tipo de Investigación:','',kind_done)
            kind_done = kind_done.strip()
            dic['research_center'].append(center)
            dic['research_type'].append(kind_done)
            dic['abstrac_esp'].append(rev_esp)
            dic['abstract_eng'].append(rev_eng)            
            dic['research_time'].append(time_done)
        for tag in tagAuthor:
            dic['autor_l'].append(tag.text.strip())
        for yr in tagYear:
            dic['year_l'].append(yr.text.strip())

    return dic

In [25]:
#For start, i will crawl the first two pages and get the dataframe from it
d =  {'research_l':[],
               'title_l':[],
               'autor_l':[],
               'year_l':[],
           'abstrac_esp':[],
           'abstract_eng':[],
           'research_center':[],
           'research_type': [],
            'research_time':[],
            'dl_link': []
           }
for i in range(0,2):
    dic = crawl_page(link,i)
    d['research_l']+=(dic['research_l'])
    d['title_l']+=(dic['title_l'])
    d['autor_l']+=(dic['autor_l'])
    d['year_l']+=(dic['year_l'])
    d['abstrac_esp']+= (dic['abstrac_esp'])
    d['abstract_eng']+= (dic['abstract_eng'])
    d['research_center']+= (dic['research_center'])
    d['research_type']+= (dic['research_type'])
    d['research_time']+= (dic['research_time'])
    d['dl_link']+= (dic['dl_link'])
    


In [26]:
#here we can see a list of the projects that won, along with their metadata.
projects_df = pandas.DataFrame.from_dict(d)
projects_df

,research_l,title_l,autor_l,year_l,abstrac_esp,abstract_eng,research_center,research_type,research_time,dl_link
0,/es/investigaciones/desarrollo-rural-medio-amb...,El impacto de la expansión de la agricultura c...,Ana Lucía Araujo Raurau,2018,Resumen del informe final: Esta investigación ...,Abstract: This research aims to analyse the im...,CEPES,Proyecto Breve,15 meses,http://www.cies.org.pe/sites/all/modules/pubdl...
1,/es/investigaciones/desarrollo-rural-medio-amb...,Derrames de petróleo y afectación a la salud m...,"Fiorella Parra, Hernán Manrique, Vania Martínez",2018,Resumen del informe final: El objetivo de esta...,Abstract: The objective of this research is to...,Universidad del Pacífico,Proyecto Breve,14 meses,http://www.cies.org.pe/sites/all/modules/pubdl...
2,/es/investigaciones/pobreza-salud/el-seguro-in...,¿El Seguro Integral de Salud reduce la vulnera...,Javier Torres,2018,Resumen del informe final: Utilizando la Encue...,Abstract: Using the National Household Survey ...,Universidad del Pacífico,Proyecto Mediano,12 meses,http://www.cies.org.pe/sites/all/modules/pubdl...
3,/es/investigaciones/base-tributaria-y-disconti...,Base tributaria y discontinuidades geopolíticas,Gabriel Natividad,2018,Resumen del informe final: \n\tEste estudio an...,Abstract: This study offers an econometric ana...,Universidad de Piura,Proyecto Corto Tributario,,http://www.cies.org.pe/sites/all/modules/pubdl...
4,/es/investigaciones/balance-sobre-el-estado-de...,Balance sobre el estado del conocimiento en po...,Alejandro Granda Sandoval,2018,Resumen del informe final: En el documento se ...,no info,,Documento de Balance,12 meses,http://www.cies.org.pe/sites/all/modules/pubdl...
5,/es/investigaciones/agenda-2030-contra-el-relo...,"Agenda 2030: ¿Contra el reloj?, la contribució...","Isabel Hurtado, Marina Irigoyen",2018,Resumen del informe final: Este documento cont...,no info,,Documentos de Política,5 meses,http://www.cies.org.pe/sites/all/modules/pubdl...
6,/es/investigaciones/boom-agroexportador-y-dive...,Auge agroexportador en el Perú: un análisis de...,Eduardo Zegarra,2018,Resumen del informe final: Se realiza un análi...,Abstract: This study presents a survival analy...,GRADE,Proyecto Mediano,10 meses,http://www.cies.org.pe/sites/all/modules/pubdl...
7,/es/investigaciones/el-seguro-integral-de-salu...,¿El Seguro Integral de Salud reduce la vulnera...,Javier Antonio Torres Gómez,2018,Resumen del informe final: \nLa presente inve...,no info,Universidad del Pacífico,Proyecto Mediano,En desarrollo,http://www.cies.org.pe/sites/all/modules/pubdl...
8,/es/investigaciones/habilidades-formacion-para...,"Habilidades, formación para el trabajo y sub-e...",Alan Sánchez,2018,Resumen del informe final: \n\tEl presente est...,Abstract: \n\tThis study uses longitudinal dat...,GRADE,Proyecto Mediano,12 meses,http://www.cies.org.pe/sites/all/modules/pubdl...
9,/es/investigaciones/cooperativismo-y-su-impact...,Cooperativismo y su impacto en el rendimiento ...,"Carlos Paredes, Kenji Moreno",2018,Resumen del informe final: \n\tEste documento ...,Abstract: \n\tThis research examines the impac...,Universidad Peruana de Ciencias Aplicadas,Proyecto Breve,11 meses,http://www.cies.org.pe/sites/all/modules/pubdl...


## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [27]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[]},"posts-start":0,"posts-total":3463,"posts-type":false,"posts":[{"id":"190233110922","url":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/190233110922","url-with-slug":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/190233110922\/a-most-fortunate-red-headed-stepkitten","type":"photo","date-gmt":"2020-01-13 11:03:57 GMT","date":"Mon, 13 Jan 2020 12:03:57","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1578913437,"format":"html","reblog-key":"BRsWJpjQ","slug":"a-most-fortunate-red-headed-stepkitten","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"190233110922\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_190233110922\"><iframe id=\"like_iframe_190233110922\" src=\"https:\/\/assets.tumblr.com\/assets\/html\/like_iframe.html?_v=66c22ab5319d742bca5762b8d18f9d06#name=

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [44]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
20


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [47]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [48]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

,id,photo-url,date,tags,photo-type
0,190233110922,https://66.media.tumblr.com/42458742dedc510f3d...,"Mon, 13 Jan 2020 12:03:57","[cat, cats, lol, lolcat, lolcats]",jpg
1,190232075547,https://66.media.tumblr.com/65b1e6efd6ec145872...,"Mon, 13 Jan 2020 10:00:35","[cat, cats, lol, lolcat, lolcats]",jpg
2,190230902981,https://66.media.tumblr.com/616b4faab5a289a369...,"Mon, 13 Jan 2020 08:01:02","[cat, cats, lol, lolcat, lolcats]",jpg
3,190229342226,https://66.media.tumblr.com/28121ad01b05305b89...,"Mon, 13 Jan 2020 06:00:57","[cat, cats, lol, lolcat, lolcats]",jpg
4,190209640041,https://66.media.tumblr.com/9b735740f517f3673b...,"Sun, 12 Jan 2020 06:01:05","[cat, cats, lol, lolcat, lolcats]",jpg
5,190193496849,https://66.media.tumblr.com/1ffa3c880a55f80f62...,"Sat, 11 Jan 2020 10:01:29","[cat, cats, lol, lolcat, lolcats]",jpg
6,190192270965,https://66.media.tumblr.com/a5ecfce3b06defdbac...,"Sat, 11 Jan 2020 08:00:47","[cat, cats, lol, lolcat, lolcats]",jpg
7,190190785530,https://66.media.tumblr.com/522386ce0644901d60...,"Sat, 11 Jan 2020 06:01:02","[cat, cats, lol, lolcat, lolcats]",jpg
8,190174143051,https://66.media.tumblr.com/5f1a4eec3910ec2262...,"Fri, 10 Jan 2020 08:00:54","[cat, cats, lol, lolcat, lolcats]",jpg
9,190153974947,https://66.media.tumblr.com/e70ac81ae13bf839f8...,"Thu, 09 Jan 2020 06:01:10","[cat, cats, lol, lolcat, lolcats]",jpg


Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [49]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [50]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. LEts tart by looking at the Shakespeare files in the `data` directory 

In [51]:
with open('../data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

FileNotFoundError: [Errno 2] No such file or directory: '../data/Shakespeare/midsummer_nights_dream.txt'

Then to load all the files in `../data/Shakespeare` we can use a for loop with `scandir`:

In [ ]:
targetDir = '../data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path) as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [ ]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [67]:
information_extraction_pdf = 'http://www.cies.org.pe/sites/default/files/investigaciones/if3_perezbarrantes_final.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])
#infoExtractionRequest.text

%PDF-1.5
%����
1 0 obj
<</Type/Catalog/Pages 2 0 R/Lang(es-PE) /StructTreeRoot 198 0 R/MarkInfo<</Marked true>>>>
endobj
2 0 obj
<</Type/Pages/Count 61/Kids[ 3 0 R 24 0 R 29 0 R 31 0 R 32 0 R 34 0 R 36 0 R 37 0 R 39 0 R 41 0 R 43 0 R 44 0 R 46 0 R 47 0 R 48 0 R 49 0 R 50 0 R 51 0 R 52 0 R 53 0 R 54 0 R 55 0 R 56 0 R 57 0 R 58 0 R 59 0 R 62 0 R 63 0 R 65 0 R 68 0 R 69 0 R 70 0 R 71 0 R 72 0 R 73 0 R 74 0 R 76 0 R 77 0 R 78 0 R 79 0 R 80 0 R 81 0 R 82 0 R 83 0 R 84 0 R 86 0 R 87 0 R 88 0 R 89 0 R 91 0 R 93 0 R 94 0 R 95 0 R 96 0 R 97 0 R 185 0 R 187 0 R 189 0 R 191 0 R 193 0 R 195 0 R] >>
endobj
3 0 obj
<</Type/Page/Parent 2 0 R/Resources<</Font<</F1 5 0 R/F2 8 0 R/F3 10 0 R/F4 15 0 R/F5 17 0 R>>/ExtGState<</GS7 7 0 R>>/XObject<</Image19 19 0 R/Image20 20 0 R/Image22 22 0 R>>/ProcSet[/PDF/Text/ImageB/ImageC/ImageI] >>/MediaBox[ 0 0 595.32 841.92] /Contents 4 0 R/Group<</Type/Group/S/Transparency/CS/DeviceRGB>>/Tabs/S/StructParents 0>>
endobj
4 0 obj
<</Filter/FlateDecode/Leng


It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [28]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [68]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [70]:
doc_for_explore = readPDF(infoExtractionBytes)

str

From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [73]:
example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

Week 1 - Retreiving and Preparing Text for Machines
This week, we begin by "begging, borrowing and stealing" text from several contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for machines to "read" and analyze. This notebook outlines scraping text from the web, from images, PDF and Word documents. Then we detail "spidering" or walking through hyperlinks to build samples of online content, and using APIs, Application Programming Interfaces, provided by webservices to access their content. Along the way, we will use regular expressions, outlined in the reading, to remove unwanted formatting and ornamentation. Finally, we discuss various text encodings, filtering and data structures in which text can be placed for analysis.
For this notebook we will be using the following packages:
#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx 

This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud_2020` so we can use it later without having to retype it.

In [74]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [75]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

File is not a zip file


We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [76]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

Week 1 - Retreiving and Preparing Text for Machines
This week, we begin by "begging, borrowing and stealing" text from several contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for machines to "read" and analyze. This notebook outlines scraping text from the web, from images, PDF and Word documents. Then we detail "spidering" or walking through hyperlinks to build samples of online content, and using APIs, Application Programming Interfaces, provided by webservices to access their content. Along the way, we will use regular expressions, outlined in the reading, to remove unwanted formatting and ornamentation. Finally, we discuss various text encodings, filtering and data structures in which text can be placed for analysis.
For this notebook we will be using the following packages:
#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx 

Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


Using the dataframe i got from section 2, i will produce a database with the content of each research document that will be analyzed. This database still needs to be separated by manageable chunks. I will be using just 2 pdf's, for start.

In [29]:
new_dic = {"whole_document":[],
          "dl_link":[]}

for link in projects_df['dl_link'][:2]:
    infoExtractionRequest = requests.get(link, stream=True)
    infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)
    doc_for_explore = readPDF(infoExtractionBytes)
    new_dic["whole_document"].append(doc_for_explore)
    new_dic["dl_link"].append(link)


### Other sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg 



In [32]:
#here we can see a list of the projects that won, along with their metadata.
pdfs_df = pandas.DataFrame.from_dict(new_dic)
new_dic

{'whole_document': [' \n \n \n \n \n \n\n \n\n  \n\n \n\nEL IMPACTO DE LA EXPANSIÓN DE LA AGRICULTURA COMERCIAL EN LA \n\nGOBERNANZA Y GESTIÓN INDÍGENA DE LA TIERRA \n\n  \n \n\nINFORME FINAL V.3 \n\nANA LUCÍA ARAUJO RAURAU \n\n \n \n\n17/12/2019 \n\n \n \n\n \n \n \n \n \n \n \n \n \n \n \n \n \nAuspicio: \n \n\n \n\n\x0cÍndice \n\nI. Diseño de investigación ......................................................................................................... 3 \n\n1.1 Planteamiento del problema ........................................................................................... 3 \n\n1.2 Marco Teórico ................................................................................................................. 6 \n\n1.3 Metodología .................................................................................................................. 10 \n\n1.4  Desarrollo del campo ...............................................................................................